---
order: 2
title: "5.2 EVER EVIL EVE"
---

> EXERCISE 5.2: EVER EVIL EVE 
> 
> Go ahead and "intercept" some of the messages encrypted by the 
> code you wrote in this section. Modify the intercepted messages 
> and verify that your decryption mechanism correctly reports an error.

--------------------------------

## Alice's Session

Alice first creates her `plaintext`. Then she uses her newly generated 
`key` and `nonce` to encrypt her `plaintext`.

In [2]:
import ex5_1
import os 

plaintext = b"Hi Bob, this is Alice!"
key = os.urandom(32)
nonce = os.urandom(16) 

encryptionManager = ex5_1.Encryptor(key=key, nonce=nonce)
ciphertextWithMac = encryptionManager.update_encryptor(plaintext=plaintext)
ciphertextWithMac += encryptionManager.finalize_encryptor()

print(f"Key: {key.hex(sep=' ')}")
print(f"Nonce: {nonce.hex(sep=' ')}")
print(f"Plaintext: {plaintext.hex(sep=' ')}")
print(f"Ciphertext With MAC: {ciphertextWithMac.hex(sep=' ')}")

Key: 0c 72 69 82 a3 54 69 16 a8 46 b1 6d 02 17 05 80 7d 1d af 4d c5 9e 6d a1 05 ff b0 fd b8 13 3e 76
Nonce: 60 8e 6d 84 0f 11 16 76 fc a3 ed ab 12 8a 4f c0
Plaintext: 48 69 20 42 6f 62 2c 20 74 68 69 73 20 69 73 20 41 6c 69 63 65 21
Ciphertext With MAC: b4 9e ac 3b c5 71 c0 e4 35 45 8b f6 21 0d b9 9c fc 48 0c 68 49 b4 8c c4 bd 0b 35 f1 fc d1 41 b6 4c 42 a9 e0 1f 83 e3 f6 86 b3 0b 5c cb 29 2e 06 31 24 91 a7 02 74


## Eve then intercepts...

Suppose Eve intercepts the `ciphertextWithMac` and she changes its first byte:

In [6]:
ciphertextWithMac = bytes.fromhex('aa') + ciphertextWithMac[1:]
print(f"Ciphertext With MAC: {ciphertextWithMac.hex(sep=' ')}")

Ciphertext With MAC: aa 9e ac 3b c5 71 c0 e4 35 45 8b f6 21 0d b9 9c fc 48 0c 68 49 b4 8c c4 bd 0b 35 f1 fc d1 41 b6 4c 42 a9 e0 1f 83 e3 f6 86 b3 0b 5c cb 29 2e 06 31 24 91 a7 02 74


## Bob, then tries to verify and decrypt the ciphertext...

We expect an `Exception` to be raised:

In [7]:
decryptionManager = ex5_1.Decryptor(
    key=key,
    nonce=nonce,
    digest=ex5_1.Decryptor.get_mac(ciphertextWithMac)
)
_plaintext = decryptionManager.update_decryptor(ciphertext=ciphertextWithMac[:-32])
_plaintext += decryptionManager.finalize_decryptor()

MessageWasTamperedWithError: 
Expected: b'\xeb\x19\xb7p`p\xe7\xba\xe5\xe3z\x81}[i\xec\xa2*\xfa\xe9]\xca\xeb\xcb\x13\x0f4\xba\x1b,\x8a\xf1'.
But got: b'\x8c\xc4\xbd\x0b5\xf1\xfc\xd1A\xb6LB\xa9\xe0\x1f\x83\xe3\xf6\x86\xb3\x0b\\\xcb).\x061$\x91\xa7\x02t'